 <span style="color:orange;font-size:20pt;font-weight:bold">CS412 - Machine Learning - Homework 2 - Berke Ayyıldızlı - 31018</span>

<span style="color:red;font-weight:bold">Required Libraries</span>

In [1]:
import numpy as np #for the calculations
import pandas as pd #to see the dataset better

<span style="font-size:14pt;">Q2. Probability and Bayes’ Theorem: Kaggle ML & DS Survey</span>

Dataset: 2017 Kaggle Machine Learning (ML) & Data Science (DS) Survey comprises over 16,000
responses from Kaggle’s industry-wide survey on the state of data science and machine learning. The dataset,
provided to you as cs412-hw2-data.zip, has 5 files:

• schema.csv: a CSV file with survey schema. This schema includes the questions that correspond to
each column name in both the multipleChoiceResponses.csv and freeformResponses.csv.

• multipleChoiceResponses.csv: Respondents’ answers to multiple choice and ranking questions.
These are non-randomized and thus a single row does correspond to all of a single user’s answers.

• freeformResponses.csv: Respondents’ freeform answers to Kaggle’s survey questions. These
responses are randomized within a column, so that reading across a single row does not give a single
user’s answers.

• conversionRates.csv: Currency conversion rates (to USD).

• RespondentTypeREADME.txt: This is a schema for decoding the responses in the ”Asked” column
of the schema.csv file.

Using the .csv files from cs412-hw2-data.zip, answer the following questions on Jupyter Notebook or
Google Colab. You do not need to add your answers to the pdf. Use a separate cell for your
code/answer for each question. You do not need to use any machine learning libraries, once you have loaded
the data you can answer all questions via Pandas and/or NumPy methods.

<span style="color:red;font-weight:bold">Reading Required Files</span>

In [2]:
#since schema, freeform responses and respondent type readme is not used during the homework, i am not reading them
questions_dataframe = pd.read_csv('multipleChoiceResponses.csv', encoding='ISO-8859-1', low_memory=False) #changed encoding and memory settings bc i kept getting errors.
currency_dataframe = pd.read_csv('conversionRates.csv', index_col=0) #deleted the index column because it was appeared as blank after transforming

<span style="color:red;">Q2.a:</span>  [5 pts] What is the probability that a respondent is currently employed as a Programmer given they
use C/C++ at work?

In [3]:
#i first filtered the users who uses c/c++ on their work, dropped the not-applicables.
c_language_users = questions_dataframe[questions_dataframe['WorkToolsSelect'].str.contains('C/C++', na=False)]

# then from the users who uses c/c++ on their work, i find the number of the "programmers".
employed_as_programmers = c_language_users[c_language_users['CurrentJobTitleSelect'].str.contains('Programmer', na=False)]
probability = len(employed_as_programmers) / len(c_language_users) #calculated the probability

########################## This part is for question Q2.g ###################################
print(len(c_language_users)/len(questions_dataframe)) #p(B)
all_programmers = questions_dataframe[questions_dataframe['CurrentJobTitleSelect'].str.contains('Programmer', na=False)]
Pa = len(all_programmers) / len(questions_dataframe)

# Print P(A)
print(f"The probability of being employed as a Programmer (P(A)): {Pa:.2%}")
########################## This part is for question Q2.g ###################################

print(f"The probability that a respondent is currently employed as a programmer given they use c/c++ at work: {probability:.2%}")#P(A|B)


0.09140942809284518
The probability of being employed as a Programmer (P(A)): 2.76%
The probability that a respondent is currently employed as a programmer given they use c/c++ at work: 2.23%


<span style="color:red">Q2.b:</span>  [5 pts] What is the probability that a respondent is a Data Scientist given they have majored in
computer science, mathematics or statistics?

In [4]:
# first i filtered the people majored in the wanted areas
majors = questions_dataframe[questions_dataframe['MajorSelect'].isin(['Computer Science', 'Mathematics or statistics'])]#p(b)

#then i selected the data scientists that are majored in these areas
data_scientists = majors[majors['CurrentJobTitleSelect'].str.contains('Data Scientist', na=False)]#p(a)
probability = len(data_scientists) / len(majors) #calculated the probability p(a|b)

print(f"The probability that a respondent is a Data Scientist given they majored in computer science, mathematics or statistics is: {probability:.2%}")


The probability that a respondent is a Data Scientist given they majored in computer science, mathematics or statistics is: 15.97%


<span style="color:red">Q2.c:</span>  [10 pts] What is the probability that a respondent works in the Technology industry given that
they earn more than 40,000 USD annually?

In [5]:
#first i merged the 2 dataframes, took CompensationCurrency from questions df and origin country from the currencies, used all the entries from the left database.
merged_dataframe = pd.merge(questions_dataframe, currency_dataframe, left_on='CompensationCurrency', right_on='originCountry', how='left')

# they earn more than 40k = p(a)
# they work in tech = p(b)

#some values had commas and lines, also coerced the errors bc some entries cannot be converted. Also without regular expressions, i got a warning.
merged_dataframe['CompensationAmount'] = pd.to_numeric(merged_dataframe['CompensationAmount'].replace(',', regex=True).replace('-', np.nan, regex=True), errors='coerce')
merged_dataframe['SalaryUSD'] = merged_dataframe['CompensationAmount'].astype(float) * merged_dataframe['exchangeRate'] #converted all salaries to usd here.

#here, i first found the amount of respondents who worked in the technology with more than 40k usd and divided it to all the respondents who earned 40k.
probability = len(merged_dataframe[merged_dataframe['SalaryUSD'] > 40000]
                  [merged_dataframe[merged_dataframe['SalaryUSD'] > 40000]
                   ['EmployerIndustry'] == 'Technology']) / len(merged_dataframe
                                                                [merged_dataframe['SalaryUSD'] > 40000]) #result of p(b|a)

print(f"The probability that a respondent works in the Technology industry given they earn more than 40,000 USD annually is: {probability:.2%}")


The probability that a respondent works in the Technology industry given they earn more than 40,000 USD annually is: 19.63%


<span style="color:red">Q2.d:</span> [5 pts] What is the joint probability of a respondent being over 30 years old and having a at
least a Bachelor’s degree?

In [6]:
#i wanted to see how many education levels there are in the column name:
print(questions_dataframe['FormalEducation'].unique())


["Bachelor's degree" "Master's degree" 'Doctoral degree' nan
 "Some college/university study without earning a bachelor's degree"
 'I did not complete any formal education past high school'
 'Professional degree' 'I prefer not to answer']


In [7]:
#took the respondents who are over 30 and has at least a bachelor degree:
#p(a) = who are over 30
#p(b) = at least has a bachelor's
#p(a^b) = (i mean a and b, not a over b)
both_criteria = questions_dataframe[(questions_dataframe['Age'] > 30) & (questions_dataframe['FormalEducation'].isin(['Bachelor’s degree', 'Master’s degree', 'Doctoral degree', 'Professional degree']))]
probability = len(both_criteria) / len(questions_dataframe)#calculated the probability

print(f"The joint probability of a respondent being over 30 years old and having at least a bachelors degree is: {probability:.2%}")


The joint probability of a respondent being over 30 years old and having at least a bachelors degree is: 12.81%


<span style="color:red">Q2.e:</span> [5 pts] What is the probability that a respondent is a Data Scientist who majored in Computer
Science, Mathematics or statistics?

In [8]:
print(questions_dataframe['CurrentJobTitleSelect'].unique()) # i wanted to see job names

['DBA/Database Engineer' nan 'Operations Research Practitioner'
 'Computer Scientist' 'Data Scientist'
 'Software Developer/Software Engineer' 'Business Analyst' 'Engineer'
 'Scientist/Researcher' 'Researcher' 'Other' 'Data Analyst'
 'Machine Learning Engineer' 'Statistician' 'Predictive Modeler'
 'Programmer' 'Data Miner']


In [9]:
data_scientists = questions_dataframe[questions_dataframe['CurrentJobTitleSelect'] == 'Data Scientist'] #all the data scientists
ds_with_wanted_majors = data_scientists[data_scientists['MajorSelect'].isin(['Computer Science', 'Mathematics or statistics'])] #data scientists with the wanted majors
probability = len(ds_with_wanted_majors) / len(questions_dataframe) #calculated the probability

print(f"The probability that a respondent is a Data Scientist who majored in Computer Science , Mathematics or Statistics is: {probability:.2%}")


The probability that a respondent is a Data Scientist who majored in Computer Science , Mathematics or Statistics is: 6.32%


<span style="color:red">Q2.f:</span>  [5 pts] What is the joint probability that a respondent is from France, earns less than 100,000
USD annually, and uses Cross-Validation Often or Most of the time?

In [10]:
users_from_france = merged_dataframe[merged_dataframe['Country'] == 'France'] #since before i merged both dataframes, i directly selected users from france, p(a)
users_earns_less = users_from_france[users_from_france['SalaryUSD'] < 100000] #users who earns less than 100 thousand dollars, p(b)

#found this class to find users who used cross validation often or most of the time p(c)
cross_validation = users_earns_less[users_earns_less['WorkMethodsFrequencyCross-Validation'].isin(['Often', 'Most of the time'])]
probability = len(cross_validation) / len(merged_dataframe) #the probability p(c^b^a)

print(f"The joint probability that a respondent is from France, earns less than 100,000 USD annually, and uses Cross-Validation Often or Most of the time is: {probability:.2%}")


The joint probability that a respondent is from France, earns less than 100,000 USD annually, and uses Cross-Validation Often or Most of the time is: 0.36%


<span style="color:red">Q2.g:</span>  [10 pts] What is the probability that a respondent uses C/C++ at work given that they are employed
as a Programmer? (Hint: Use your findings from Question 2a).

In [11]:
#THE FIRST METHOD (Calculating Again)

# users who are employed as programmers:
programmers = questions_dataframe[questions_dataframe['CurrentJobTitleSelect'].str.contains('Programmer', na=False)]

# from programmers, find users who use c++
programmers_using_c = programmers[programmers['WorkToolsSelect'].str.contains('C/C++', na=False)]

probability = len(programmers_using_c) / len(programmers)# the probability

print(f"The probability that a respondent uses C/C++ at work given that they are employed as a Programmer is: {probability:.2%}")


The probability that a respondent uses C/C++ at work given that they are employed as a Programmer is: 7.36%


THE SECOND METHOD (Bayes Theorem)

Since from the question 2a, we have

P(A) = The probability of a respondent that is currently employed as a Programmer = 0.027

P(B) = The probability that they use C/C++ at work = 0.091

We calculated: P(A|B) = 0.0223 = 2.23%

Here we need to calculate P(B|A) = (P(A|B) x P(B)) / P(A)

P(B|A) = (0.0223 x 0.091) / 0.027 = 0,075 = 7.5%

We can see that the values are the same (the small difference is because of the decimal points), thus making the Bayes theorem valid.


<span style="color:red">Q2.h:</span>  [15 pts] Given the probability of a respondent wearing glasses is 0.15, and the probability of a
respondent wearing glasses given they have a PhD is 0.25, find the probability of a respondent having a PhD
given that they wear glasses.

P(B)=0.15 (the probability of wearing glasses).

P(B|A)=0.25 (the probability of wearing glasses given that the respondent has a PhD).

We need to find P(A|B) (the probability of having a PhD given that the respondent wears glasses).

In [12]:
# in this question we need to find the p(A), meaning the probability of having a doctorate.
dropped_dataframe = questions_dataframe.dropna(subset=['FormalEducation']) #i started dropping not-applicable's and I prefer not to answer
# used shape[] to give the total number of rows (count of doctoral degrees)
doctorate_count = dropped_dataframe[dropped_dataframe['FormalEducation'] == 'Doctoral degree'].shape[0]
all_count = dropped_dataframe.shape[0] #number of responses

# calculated the probability of having a PhD
probability_phd = doctorate_count / all_count

print(f"P(A): {probability_phd:.2%}")


P(A): 15.63%


Since %15.63 is equals to 0.1563, we can plug all the values into the Bayes theorem:

P(A|B) = (P(B|A) x P (A)) / P(B) = ((0.25) x (0.1563)) / (0.15) = % 26.05

Giving us the result of 0.2605 or %26.05 of the probability of a respondent having a phd given they wear glasses.

This is the end of the Homework 2 Question 2.